In [1]:
import sys
import os
import numpy as np

from tqdm import tqdm
import numba
import datetime

In [2]:
# vor_path = "/home/ubuntu/git/active_vertex"
vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'
sys.path.append(vor_path)

import voronoi_model.voronoi_model_periodic as avm

In [3]:
def print_SPV_params(vor2, p, J):
    print(
        """
    Number of cells : {0}
    Time-span       : {1:.2f} - {2:.2f}
    Time-step (dt)  : {3:.3f}

    === Parameters ===
    Dr      : {4:.1e}
    kappa_A : {5:.2f}
    kappa_P : {6:.2f}
    a       : {7:.2f}
    k       : {8:.1f}
    p0      : {9:.2f}
    v0      : {10:.2e}
    J (adh) : {11:.3f}
    """.format(
            vor2.n_c,
            vor2.t_span[0],
            vor2.t_span[-1],
            vor2.dt,
            vor2.Dr,
            vor2.kappa_A,
            vor2.kappa_P,
            vor2.a,
            vor2.k,
            p,
            vor2.v0,
            J
        )
    )

In [4]:
@numba.njit
def get_D_eff(X0, Xmax, L, tmax, v0, Dr, n=19):
    
    X0_norm = X0 - L/2
    center_cells = np.argsort(np.sqrt(X0_norm[:, 0]**2 + X0_norm[:, 1]**2))[:n]
    
    dX = np.fmod(Xmax[center_cells] - X0[center_cells], L/2)
    dr2 = np.zeros(n, dtype=np.float32)
    for i in range(n):
        dr2[i] = dX[i, 0] ** 2 + dX[i, 1] ** 2
    
    Ds = dr2.mean() / (4*tmax)
    D0 = v0**2/(2*Dr)
    return Ds / D0

def vor_to_D_eff(vor2, n=7):
    return get_D_eff(
        vor2.x_save[0],
        vor2.x_save[-1],
        vor2.L,
        vor2.tfin,
        vor2.v0,
        vor2.Dr,
        n=n,
    )


In [5]:
p_space = np.linspace(3.5, 3.9, 5)
J_space = [0]
v_space = np.logspace(-2, 0, 5)[1:]

param_space = np.meshgrid(
    p_space, 
    J_space,
    v_space,
)
param_space = np.array(param_space).T.reshape(-1, 3)

In [6]:
param_space.shape

(20, 3)

In [52]:
param_space = np.array([
    [3.5e+00, 0.0e+00, 1.0e-03],
    [3.5e+00, 0.0e+00, 5.0e-03],
    [3.5e+00, 0.0e+00, 1.0e-02],
    [3.5e+00, 0.0e+00, 5.0e-02],
    [3.5    , 0.     , 0.02   ],
    [3.5    , 0.     , 0.03   ],
    [3.5    , 0.     , 0.04   ],
    [3.5    , 0.     , 0.01   ],
    [3.6    , 0.     , 0.01   ],
    [3.7    , 0.     , 0.01   ],
    [3.8    , 0.     , 0.01   ],
])


In [74]:
f = 200
t0 = 0
tmax = 1
dt = 0.05
n_t = int((tmax - t0) * f / dt) + 1  # calculates the n_t to get the desired dt

a = 0.35
k = 2

In [75]:
D_eff_save = []

iterator = param_space[1:]
# iterator = tqdm(iterator)

for p, J, v in iterator:
    prefix = f"p0{p:.2f}_J{J:.2f}_v0{v:.2e}_big"
    print(prefix)
    
    vor2 = avm.Tissue()
    vor2.generate_cells(600)
    vor2.make_init(18)

    vor2.set_GRN_t_span(t0, tmax, n_t, scaling_factor=f);
    vor2.v0 = v
#     vor2.n_warmup_steps = int(150 / dt)
    vor2.n_warmup_steps = 1

    W = J * np.array([[1, 0], [0, 1]])
    vor2.set_interaction(W=W, pE=0)
    vor2.A0 = 0.86
    vor2.P0 = p * np.sqrt(vor2.A0)

    vor2.Dr = 0.01
    vor2.kappa_A = 0.2
    vor2.kappa_P = 0.1
    vor2.a = a
    vor2.k = k
    
    vor2.simulate(progress_bar=True, print_updates=True);
    
    D_eff =vor_to_D_eff(vor2, n=37)
    print(D_eff)
    D_eff_save.append(D_eff)
    
    vor2.animate(file_name=prefix, n_frames=100, print_updates=False)

p03.50_J0.00_v05.00e-03_big
Warming up SPV (1 steps)


100%|██████████| 4000/4000 [05:31<00:00, 12.05it/s]


0.013425454497337341
p03.50_J0.00_v01.00e-02_big
Warming up SPV (1 steps)


100%|██████████| 4000/4000 [05:32<00:00, 12.02it/s]


0.0019141134107485414
p03.50_J0.00_v05.00e-02_big
Warming up SPV (1 steps)


 38%|███▊      | 1531/4000 [02:08<03:27, 11.90it/s]


KeyboardInterrupt: 

In [47]:
param_space

array([[3.5 , 0.  , 0.01],
       [3.6 , 0.  , 0.01],
       [3.7 , 0.  , 0.01],
       [3.8 , 0.  , 0.01]])

In [ ]:
D_eff_save

It looks like `v0 = 0.01` is similar to `v = 0.5` in the source publication: 
- https://journals.aps.org/prx/abstract/10.1103/PhysRevX.6.021011

In [38]:
# For these params
ex_params = np.array([
    [3.5e+00, 0.0e+00, 1.0e-03],
    [3.5e+00, 0.0e+00, 5.0e-03],
    [3.5e+00, 0.0e+00, 1.0e-02],
    [3.5e+00, 0.0e+00, 5.0e-02],
    [3.5    , 0.     , 0.02   ],
    [3.5    , 0.     , 0.03   ],
    [3.5    , 0.     , 0.04   ],
    [3.5    , 0.     , 0.01   ],
    [3.6    , 0.     , 0.01   ],
    [3.7    , 0.     , 0.01   ],
    [3.8    , 0.     , 0.01   ],
])

# I got these D_eff values
ex_D_eff = np.array([
    0.005111410791869276,
    0.01961112953722477,
    0.014301142655313015,
    0.025674479007720942,
    0.012277007102966307, 
    0.01292151129908032, 
    0.003912175539880991,   # wat
    0.03499118611216545,
    0.007614209782332182,
    0.04667229205369949,
    0.09706597030162811,
])